In [ ]:
# Jinseok Ryu
# jinseuk56@gmail.com

import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hys
import tifffile as tff
import tkinter.filedialog as tkf
import ipywidgets as pyw
from tabulate import tabulate
from sklearn.cluster import DBSCAN
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
%matplotlib inline

In [ ]:
num_split = 3
adr = []
num_adr = []
for i in range(num_split):
    temp = tkf.askopenfilenames()
    num_adr.append(len(temp))
    adr.append(temp)
print(num_adr)
num_img = np.max(num_adr)

In [ ]:
#rel_lmb = 0.00197 # nm, relativistic wavelength for 300 kV
#rel_lmb = 0.00251 # nm, relativistic wavelength for 200 kV
k_per_pixel = 0.1722 # 1/nm
r_per_pixel = 2.4416 # nm

s_dim = 60
cb_rad = 7
k_list = np.arange(s_dim) * k_per_pixel
print(k_list)

In [ ]:
data_by_split = []
data_flat = []
data_flat_by_split = []
data_shape = []
for i in range(num_split):
    temp = []
    temp_shape = []
    temp_flat = []
    for path in adr[i]:
        print(path)
        tmp = hys.load(path).data
        print(tmp.shape)
        if len(np.where(tmp[0, 0] == 0.0)[0]) != cb_rad:
            tmp = np.insert(tmp, 0, 0, axis=2)        
        
        tmp[:, :, :cb_rad] = 0.0
        tmp = tmp[:, :, :s_dim]
        print(tmp.shape)
        temp.append(tmp)
        temp_shape.append(tmp.shape)
        temp_flat.extend(tmp.reshape(-1, tmp.shape[2]).tolist())
        data_flat.extend(tmp.reshape(-1, tmp.shape[2]).tolist())
    data_by_split.append(temp)
    data_shape.append(temp_shape)
    data_flat_by_split.append(temp_flat)

In [ ]:
data_flat = np.asarray(data_flat)
print(data_flat.shape)
data_shape = np.asarray(data_shape)
print(data_shape.shape)
print(s_dim)

In [ ]:
radial_var_sum = []
for arr in data_by_split:
    temp = []
    for img in arr:
        temp.append(np.mean(img, axis=(0, 1)))
    radial_var_sum.append(temp)

In [ ]:
color_rep = ["black", "gray", "green", "red", "orange", "blue", "purple", "hotpink", "lime", "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", "yellow"]
print(len(color_rep))

In [ ]:
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.tick_params(axis="x", labelsize=15)
for i in range(num_split):
    for line in radial_var_sum[i]:
        ax.plot(k_list, line, color=color_rep[i+1], linestyle="-", label="split %d"%(i+1))
#ax.grid()
#fig.legend(fontsize="xx-large")
fig.tight_layout()
plt.show()

In [ ]:
k_ranges = [[21, 21], [40, 40]]
for a, b in k_ranges:
    print(k_list[a])
radial_var_split = []
for i in range(num_split):
    radial_var_split.append(np.mean(data_flat_by_split[i], axis=0))
    
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.tick_params(axis="x", labelsize=15)
for i in range(num_split):
    ax.plot(radial_var_split[i], color=color_rep[i+1], linestyle="-", label="split %d"%(i+1))
    print(np.max(radial_var_split[i]))

for rg in k_ranges:
    ax.fill_between([rg[0], rg[1]], np.max(radial_var_split), alpha=0.5, color="orange")
#ax.grid()
#fig.legend(fontsize="xx-large")
fig.tight_layout()
plt.show()

In [ ]:
radial_var_split = np.asarray(radial_var_split)
max_var = radial_var_split[:, 21]
print(max_var.shape)

mean_rel_thick = [0.275,  0.592, 0.491]

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
#ax[0].plot(mean_rel_thick, radial_var_split[:, 21], 'k-')
ax[0].plot(mean_rel_thick, radial_var_split[:, 21], 'r*')
#ax[1].plot(mean_rel_thick, radial_var_split[:, 35], 'k-')
ax[1].plot(mean_rel_thick, radial_var_split[:, 35], 'r*')

ax[0].tick_params(axis="both", labelsize=15)
ax[1].tick_params(axis="both", labelsize=15)
fig.tight_layout()
plt.show()

In [ ]:
sel_range = 1

var_imgs = []
max_vals = []
min_vals = []
for i in range(num_split):
    for arr in data_by_split[i]:
        tmp = np.sum(arr[:, :, k_ranges[sel_range][0]:(k_ranges[sel_range][1])], axis=2)
        max_vals.append(np.max(tmp))
        min_vals.append(np.min(tmp))
        var_imgs.append(tmp)
        
min_val = np.min(min_vals)
max_val = np.max(max_vals)

for im in var_imgs:
    fig, ax = plt.subplots(1, 1)
    ax.imshow(im, vmin=min_val, vmax=max_val, cmap="inferno")
    ax.axis("off")
    fig.tight_layout()
    plt.show()

In [ ]:
# assume the shape of clusters is circle
def cluster_size(img):
    label = np.unique(img)
    label = label[label != -1]
    #print(label)
    cluster_size = []
    for i in label:
        cluster_size.append(len(np.where(img==i)[0]))
    cluster_size = np.sort(cluster_size)
    #print(cluster_size)

    largest = 2*np.sqrt(cluster_size[-1]/np.pi)
    smallest = 2*np.sqrt(cluster_size[0]/np.pi)
    avg = 2*np.sqrt(np.mean(cluster_size)/np.pi)
    avg_peri = 2*np.sqrt(np.mean(cluster_size)*np.pi)
    
    return avg, avg_peri, largest, smallest

In [ ]:
%matplotlib widget

fig3, ax3 = plt.subplots(2, 2, figsize=(20, 6))
row_label = ["area (%)", "# of clusters (per 10000 nm^2)", "mean size (nm)", "largest size (nm)", "smallest size (nm)"]

def selector(k1, k2, binsize, m_sigma, m, e, split, n_img, save_option):
    db = DBSCAN(eps=e, min_samples=m)
    ind = num_img*(split-1)+n_img-1

    ax3[0][0].cla()
    ax3[0][1].cla()
    ax3[1][0].cla()
    ax3[1][1].cla()

    
    for i in range(num_split):
        ax3[0][0].plot(radial_var_split[i], color=color_rep[i+1], linestyle="-", label="split %d"%(i+1))
    
    flat = np.array(data_by_split[split-1][n_img-1])[:, :, k1:(k2+1)].flatten()
    #binsize = int(0.005*len(flat))
    #print("binning size (histogram) = %d"%binsize)
    ax3[0][1].hist(flat, bins=binsize, color=color_rep[split], label="split %d"%(split), histtype="step")
    
    print("left k = %f"%(k1*k_per_pixel))
    print("right k = %f"%(k2*k_per_pixel))
    
    histo, hist_edges = np.histogram(flat, bins=binsize)
    hist_edges = hist_edges[1:]
    grad = np.gradient(histo)
    second_order = np.gradient(grad)
    #zero_closest = np.argsort(np.square(second_order[:int(binsize/2)]))
    extreme_point = np.argmin(grad)
    
    ax3[0][1].plot(hist_edges, grad, "k+")
    ax3[0][1].plot(hist_edges, second_order, "r-")
    
    v1 = np.mean(flat) + m_sigma * np.std(flat)
    v2 = np.percentile(flat, 100)
    
    ax3[0][0].fill_between([k1, k2], np.max(radial_var_split), alpha=0.5, color="orange")
    ax3[0][0].grid()
    #ax3[0].legend(fontsize="xx-large")
    
    ax3[0][1].fill_between([v1, v2], ax3[0][1].get_ylim()[1], alpha=0.5, color="orange")
    ax3[0][1].grid()
      
    img = np.array(data_by_split[split-1][n_img-1])[:, :, k1:(k2+1)]
    img[np.where(img < v1)] = 0
    img[np.where(img > v2)] = 0
    img[np.where(img != 0)] = 1
    bool_img = np.sum(img, axis=2)
    bool_img = np.divide(bool_img, np.where(bool_img!=0, bool_img, 1))
    ax3[1][0].imshow(bool_img, cmap="afmhot")
    ax3[1][0].axis("off")
    ax3[1][0].set_title("%.2f %%"%(np.sum(bool_img)*100/(img.shape[0]*img.shape[1])))
        
    sel_coor = np.nonzero(bool_img)
    X = np.stack((sel_coor[0], sel_coor[1]), axis=1)
    db.fit(X)
    label = db.labels_
    img_tmp = np.zeros_like(bool_img)
    img_tmp += -1
    img_tmp[X[:, 0], X[:, 1]] = label
    cluster_img = np.where(img_tmp<0, img_tmp, 1)
    cluster_img = np.where(cluster_img>-1, bool_img, 0)
    #ax3[3].set_title("split %d, image %d, epsilon=%.1f, min_#_of_samples=%d"%(split, n_img, e, m), fontsize=10)
    ax3[1][1].imshow(cluster_img, cmap="afmhot")
    ax3[1][1].axis("off")
    
    area = np.around(100 * np.sum(cluster_img) / (img_tmp.shape[0]*img_tmp.shape[1]), decimals=2)
    per_10000sqnm = 10000 / (img_tmp.shape[0]*img_tmp.shape[1]*r_per_pixel**2)
    n_cluster = np.around((len(np.unique(label)) - 1) * per_10000sqnm, decimals=0)
    if n_cluster != 0:
        avg, avg_peri, largest, smallest = cluster_size(img_tmp)
        avg = np.around(avg*r_per_pixel, decimals=2)
        avg_peri = np.around(avg_peri*r_per_pixel, decimals=2)
        largest = np.around(largest*r_per_pixel, decimals=2)
        smallest = np.around(smallest*r_per_pixel, decimals=2)
        table_vals = [[area], [n_cluster], [avg], [largest], [smallest]]
        print("----------------------------------------")
        print("split %d, image %d"%(split, n_img))
        print("epsilon=%.1f, minimum # of samples=%d"%(e, m))
        print(tabulate([["area (%)", area], ["# of cluster (per 10000 nm^2)", n_cluster], ["mean perimeter", avg_peri], 
                        ["mean size", avg], ["largest size", largest], ["smallest size", smallest]]))
        
    else:
        print("----------------------------------------")
        print("there is no clusters")

    if save_option:
        tff.imsave(tkf.asksaveasfilename(), cluster_img)
    
    fig3.tight_layout()

st = {"description_width": "initial"}    
k1_widget = pyw.IntText(value=28, description="left of k-range", style=st)
k2_widget = pyw.IntText(value=28, description="right of k-range", style=st)
bin_size = pyw.IntText(value=50, description="histogram bin size", style=st)
sigma_widget = pyw.FloatText(value=2.0, description="sigma: ", style=st)
min_widget = pyw.IntText(value=4, description="minimum # of cluster pixels", style=st)
eps_widget = pyw.FloatText(value=1.5, description="epsilon: ", style=st)
split_widget = pyw.Dropdown(options=np.arange(num_split)+1, value = 1, description="split: ", style=st)
n_img_widget = pyw.Dropdown(options=np.arange(np.max(num_adr))+1, value = 1, description="image: ", style=st)
save_widget = pyw.Checkbox(value=False, description="save the result")

pyw.interact(selector, k1=k1_widget, k2=k2_widget, binsize=bin_size, m_sigma=sigma_widget, m=min_widget, e=eps_widget, 
             split=split_widget, n_img=n_img_widget, save_option=save_widget)
fig3.show()

In [ ]:
%matplotlib widget
binsize = 2000
fig1, ax1 = plt.subplots(1, 2, figsize=(15, 6))
fig2, ax2 = plt.subplots(num_split, num_img, figsize=(15, 15))
selected_img = []

def var_selector(k1, k2, p1, p2):
    
    v1 = np.percentile(data_flat[:, k1:(k2+1)], p1)
    v2 = np.percentile(data_flat[:, k1:(k2+1)], p2)
    
    print("left k = %f"%(k1*k_per_pixel))
    print("right k = %f"%(k2*k_per_pixel))
    print("left variance = %f"%v1)
    print("right variance = %f"%v2)
    
    ax1[0].cla()
    ax1[1].cla()
    selected_img.clear()
    for i in range(num_split):
        ax1[0].plot(radial_var_split[i], color=color_rep[i+1], linestyle="-", label="split %d"%(i+1))
        ax1[1].hist(np.array(data_flat_by_split[i])[:, k1:(k2+1)].flatten(), bins=binsize, 
                    color=color_rep[i+1], alpha=0.5, label="split %d"%(i+1), density=True, histtype="step")
        for j in range(num_img):
            img = np.array(data_by_split[i][j])[:, :, k1:(k2+1)]
            img[np.where(img < v1)] = 0
            img[np.where(img > v2)] = 0
            img[np.where(img != 0)] = 1
            bool_img = np.sum(img, axis=2)
            bool_img = np.divide(bool_img, np.where(bool_img!=0, bool_img, 1))
            selected_img.append(bool_img)
            ind = num_img*i+j
            #print(ind)
            ax2.flat[ind].imshow(bool_img, cmap="afmhot")
            ax2.flat[ind].axis("off")
            ax2.flat[ind].set_title("%.2f %%"%(np.sum(bool_img)*100/(img.shape[0]*img.shape[1])))
                
    ax1[0].fill_between([k1, k2], np.max(radial_var_split), alpha=0.5, color="orange")
    ax1[0].grid()
    ax1[0].legend(fontsize="xx-large")
    ax1[1].fill_between([v1, v2], ax1[1].get_ylim()[1], alpha=0.5, color="orange")
    ax1[1].grid()
    ax1[1].legend(fontsize="xx-large")
    
    fig1.canvas.draw()
    fig2.canvas.draw()
    fig1.tight_layout()
    fig2.tight_layout()
    
st = {"description_width": "initial"}
k1_widget = pyw.IntText(value=28, description="left of k-range", style=st)
k2_widget = pyw.IntText(value=28, description="right of k-range", style=st)
p1_widget = pyw.FloatText(value=90.0, description="left percentile of variance range", style=st)
p2_widget = pyw.FloatText(value=99.999, description="right percentile of variance range", style=st)
pyw.interact(var_selector, k1=k1_widget, k2=k2_widget, p1=p1_widget, p2=p2_widget)
plt.show()

In [ ]:
k_range = [k1_widget.value, k2_widget.value]
print(k_range)
v_range = [np.percentile(data_flat, p1_widget.value), np.percentile(data_flat, p2_widget.value)]
print(v_range)

In [ ]:
%matplotlib widget
#fig3, ax3 = plt.subplots(1, 3, figsize=(20, 10))
fig3, ax3 = plt.subplots(1, 2, figsize=(5, 10))
row_label = ["area (%)", "# of clusters (per 10000 nm^2)", "mean size (nm)", "largest size (nm)", "smallest size (nm)"]

def selector(m, e, split, n_img, save_option):
    db = DBSCAN(eps=e, min_samples=m)
    ind = num_img*(split-1)+n_img-1

    ax3[0].cla()
    ax3[1].cla()
    #ax3[2].cla()
    
    ax3[0].imshow(selected_img[ind], cmap="afmhot")
    ax3[0].set_title("%.2f %%"%(np.sum(selected_img[ind])*100/(selected_img[ind].shape[0]*selected_img[ind].shape[1])))
    ax3[0].axis("off")  
    
    sel_coor = np.nonzero(selected_img[ind])
    X = np.stack((sel_coor[0], sel_coor[1]), axis=1)
    db.fit(X)
    label = db.labels_
    img_tmp = np.zeros_like(selected_img[ind])
    img_tmp += -1
    img_tmp[X[:, 0], X[:, 1]] = label
    bool_img = np.where(img_tmp<0, img_tmp, 1)
    bool_img = np.where(bool_img>-1, bool_img, 0)
    #ax3[1].set_title("split %d, image %d, epsilon=%.1f, min_#_of_samples=%d"%(split, n_img, e, m), fontsize=15)
    ax3[1].imshow(bool_img, cmap="afmhot")
    ax3[1].axis("off")
    
    area = np.around(100 * np.sum(bool_img) / (img_tmp.shape[0]*img_tmp.shape[1]), decimals=2)
    print("area of the roi = %.2f"%(img_tmp.shape[0]*img_tmp.shape[1]*r_per_pixel**2))
    per_10000sqnm = 10000 / (img_tmp.shape[0]*img_tmp.shape[1]*r_per_pixel**2)
    n_cluster = np.around((len(np.unique(label)) - 1) * per_10000sqnm, decimals=0)
    if n_cluster != 0:
        avg, avg_peri, largest, smallest = cluster_size(img_tmp)
        avg = np.around(avg*r_per_pixel, decimals=2)
        avg_peri = np.around(avg_peri*r_per_pixel, decimals=2)
        largest = np.around(largest*r_per_pixel, decimals=2)
        smallest = np.around(smallest*r_per_pixel, decimals=2)
        table_vals = [[area], [n_cluster], [avg], [largest], [smallest]]
        print("----------------------------------------")
        print("split %d, image %d"%(split, n_img))
        print("epsilon=%.1f, minimum # of samples=%d"%(e, m))
        print(tabulate([["area (%)", area], ["# of cluster (per 10000 nm^2)", n_cluster], ["mean perimeter", avg_peri], 
                        ["mean size", avg], ["largest size", largest], ["smallest size", smallest]]))
        #table = ax3[2].table(cellText=table_vals, rowLabels=row_label, loc="center", 
        #                     rowLoc="center", cellLoc="center", colWidths=[0.1]*3)
        #table.scale(3, 8)
        #table.auto_set_font_size(False)
        #table.set_fontsize(15)
        
    else:
        print("----------------------------------------")
        print("there is no clusters")


    #ax3[2].axis("off")
    fig3.tight_layout()
    
    if save_option:
        tff.imsave(tkf.asksaveasfilename(), bool_img)
    
st = {"description_width": "initial"}
min_widget = pyw.IntText(value=4, description="minimum # of cluster pixels", style=st)
eps_widget = pyw.FloatText(value=1.5, description="epsilon: ")
split_widget = pyw.Dropdown(options=np.arange(num_split)+1, value = 1, description="split: ", style=st)
n_img_widget = pyw.Dropdown(options=np.arange(np.max(num_adr))+1, value = 1, description="image: ", style=st)
save_widget = pyw.Checkbox(value=False, description="save the result")

pyw.interact(selector, m=min_widget, e=eps_widget, split=split_widget, n_img=n_img_widget, save_option=save_widget)
fig3.show()

In [ ]:
%matplotlib widget
binsize = 2000
#fig3, ax3 = plt.subplots(1, 5, figsize=(20, 6))
fig3, ax3 = plt.subplots(1, 4, figsize=(20, 6))
row_label = ["area (%)", "# of clusters (per 10000 nm^2)", "mean size (nm)", "largest size (nm)", "smallest size (nm)"]

def selector(k1, k2, p1, p2, m, e, split, n_img, save_option):
    db = DBSCAN(eps=e, min_samples=m)
    ind = num_img*(split-1)+n_img-1

    ax3[0].cla()
    ax3[1].cla()
    ax3[2].cla()
    ax3[3].cla()
    #ax3[4].cla()
    
    for i in range(num_split):
        ax3[0].plot(radial_var_split[i], color=color_rep[i+1], linestyle="-", label="split %d"%(i+1))
    
    flat = np.array(data_by_split[split-1][n_img-1])[:, :, k1:(k2+1)].flatten()
    ax3[1].hist(flat, bins=binsize, color=color_rep[split], label="split %d"%(split), density=True, histtype="step")
        
    print("left k = %f"%(k1*k_per_pixel))
    print("right k = %f"%(k2*k_per_pixel))
    
    v1 = np.percentile(flat, p1)
    v2 = np.percentile(flat, p2)
    print("left variance = %f"%v1)
    print("right variance = %f"%v2)
    
    ax3[0].fill_between([k1, k2], np.max(radial_var_split), alpha=0.5, color="orange")
    ax3[0].grid()
    #ax3[0].legend(fontsize="xx-large")
    ax3[1].fill_between([v1, v2], ax3[1].get_ylim()[1], alpha=0.5, color="orange")
    ax3[1].grid()    
    
    img = np.array(data_by_split[split-1][n_img-1])[:, :, k1:(k2+1)]
    img[np.where(img < v1)] = 0
    img[np.where(img > v2)] = 0
    img[np.where(img != 0)] = 1
    bool_img = np.sum(img, axis=2)
    bool_img = np.divide(bool_img, np.where(bool_img!=0, bool_img, 1))
    ax3[2].imshow(bool_img, cmap="afmhot")
    ax3[2].axis("off")
    ax3[2].set_title("%.2f %%"%(np.sum(bool_img)*100/(img.shape[0]*img.shape[1])))
        
    sel_coor = np.nonzero(bool_img)
    X = np.stack((sel_coor[0], sel_coor[1]), axis=1)
    db.fit(X)
    label = db.labels_
    img_tmp = np.zeros_like(bool_img)
    img_tmp += -1
    img_tmp[X[:, 0], X[:, 1]] = label
    cluster_img = np.where(img_tmp<0, img_tmp, 1)
    cluster_img = np.where(cluster_img>-1, bool_img, 0)
    #ax3[3].set_title("split %d, image %d, epsilon=%.1f, min_#_of_samples=%d"%(split, n_img, e, m), fontsize=10)
    ax3[3].imshow(cluster_img, cmap="afmhot")
    ax3[3].axis("off")
    
    area = np.around(100 * np.sum(cluster_img) / (img_tmp.shape[0]*img_tmp.shape[1]), decimals=2)
    per_10000sqnm = 10000 / (img_tmp.shape[0]*img_tmp.shape[1]*r_per_pixel**2)
    n_cluster = np.around((len(np.unique(label)) - 1) * per_10000sqnm, decimals=0)
    if n_cluster != 0:
        avg, avg_peri, largest, smallest = cluster_size(img_tmp)
        avg = np.around(avg*r_per_pixel, decimals=2)
        avg_peri = np.around(avg_peri*r_per_pixel, decimals=2)
        largest = np.around(largest*r_per_pixel, decimals=2)
        smallest = np.around(smallest*r_per_pixel, decimals=2)
        table_vals = [[area], [n_cluster], [avg], [largest], [smallest]]
        print("----------------------------------------")
        print("split %d, image %d"%(split, n_img))
        print("epsilon=%.1f, minimum # of samples=%d"%(e, m))
        print(tabulate([["area (%)", area], ["# of cluster (per 10000 nm^2)", n_cluster], ["mean perimeter", avg_peri], 
                        ["mean size", avg], ["largest size", largest], ["smallest size", smallest]]))
        
    else:
        print("----------------------------------------")
        print("there is no clusters")

    if save_option:
        tff.imsave(tkf.asksaveasfilename(), cluster_img)
    
    fig3.tight_layout()

st = {"description_width": "initial"}    
k1_widget = pyw.IntText(value=28, description="left of k-range", style=st)
k2_widget = pyw.IntText(value=28, description="right of k-range", style=st)
p1_widget = pyw.FloatText(value=90.0, description="left percentile of variance range", style=st)
p2_widget = pyw.FloatText(value=99.999, description="right percentile of variance range", style=st)

min_widget = pyw.IntText(value=4, description="minimum # of cluster pixels", style=st)
eps_widget = pyw.FloatText(value=1.5, description="epsilon: ", style=st)
split_widget = pyw.Dropdown(options=np.arange(num_split)+1, value = 1, description="split: ", style=st)
n_img_widget = pyw.Dropdown(options=np.arange(np.max(num_adr))+1, value = 1, description="image: ", style=st)
save_widget = pyw.Checkbox(value=False, description="save the result")

pyw.interact(selector, k1=k1_widget, k2=k2_widget, p1=p1_widget, p2=p2_widget, m=min_widget, e=eps_widget, 
             split=split_widget, n_img=n_img_widget, save_option=save_widget)
fig3.show()